In [20]:
import boto3, json, numpy as np
import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1) Your pancake text (again)
doc_text = """
How to Make Simple Pancakes:

Ingredients:
– 1 cup all-purpose flour  
– 2 tablespoons sugar  
– 1 teaspoon baking powder  
– 1 teaspoon baking soda  
– 1 pinch of salt  
– 1 cup buttermilk  
– 1 egg  
– 2 tablespoons melted butter  

Instructions:
In a bowl, whisk together the flour, sugar, baking powder, baking soda, and salt.  
In another bowl, beat the egg with the buttermilk and melted butter until combined.  
Pour the wet ingredients into the dry ingredients and stir until just mixed—it's okay if there are a few lumps.  

Cooking:
Heat a non-stick skillet over medium heat and lightly grease with butter.  
Pour ¼ cup batter per pancake; cook until bubbles form on top (about 2 minutes), then flip and cook the other side until golden brown.  
Serve warm with maple syrup, fresh fruit, or your favorite toppings.
"""

# 2) Chunk into ~200-char slices with 50-char overlap
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n"," \n"," ",""],
    chunk_size=200,
    chunk_overlap=50
)
chunks = splitter.split_text(doc_text)

# 3) Bedrock runtime client
rt = boto3.client("bedrock-runtime", region_name="us-east-1")

# 4) Embed one chunk at a time (using inputText)
vectors = []
for i, chunk in enumerate(chunks):
    resp = rt.invoke_model(
      modelId=EMBED_MODEL_ID,
      contentType="application/json",
      accept="application/json",
      body=json.dumps({"inputText": chunk})
    )
    data = json.loads(resp["body"].read())
    vectors.append(data["embedding"])
    # optional progress print
    if (i+1) % 5 == 0 or i==len(chunks)-1:
        print(f"Embedded chunk {i+1}/{len(chunks)}")

# 5) Normalize & index in FAISS
arr = np.array(vectors, dtype="float32")
arr /= np.linalg.norm(arr, axis=1, keepdims=True)

index = faiss.IndexFlatIP(arr.shape[1])
index.add(arr)

print("✅ Built FAISS index with", index.ntotal, "vectors")


Embedded chunk 5/8
Embedded chunk 8/8
✅ Built FAISS index with 8 vectors


In [21]:
# ─── Cell A ───
# Replace with the model you see “Access granted” for in your Base models list
TEXT_MODEL_ID = "amazon.titan-text-express-v1"
print("🔑 Using TEXT_MODEL_ID =", TEXT_MODEL_ID)


🔑 Using TEXT_MODEL_ID = amazon.titan-text-express-v1


In [22]:
import json, numpy as np

# Make sure you have already set:
#   EMBED_MODEL_ID = "amazon.titan-embed-g1-text-02"
#   TEXT_MODEL_ID = "amazon.titan-text-express-v1"  # the Express G1 you opted into

# 1) Retrieval helper—unchanged
def retrieve(question: str, k: int = 2) -> list[str]:
    resp = rt.invoke_model(
        modelId=EMBED_MODEL_ID,
        contentType="application/json",
        accept="application/json",
        body=json.dumps({"inputText": question})
    )
    vec = np.array(json.loads(resp["body"].read())["embedding"], dtype="float32")
    vec /= np.linalg.norm(vec)
    _, ids = index.search(vec.reshape(1, -1), k)
    return [chunks[i] for i in ids[0]]

# 2) RAG + Titan-Text-Express invoke
def rag(question: str, k: int = 2) -> str:
    ctx    = "\n\n---\n\n".join(retrieve(question, k))
    prompt = f"Context:\n{ctx}\n\nQuestion: {question}"

    payload = {
      "inputText": prompt,
      "textGenerationConfig": {
         "temperature": 0.0,
         "topP": 1.0,
         "maxTokenCount": 256    # omit stopSequences entirely
      }
    }

    resp = rt.invoke_model(
        modelId=TEXT_MODEL_ID,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(payload)
    )
    result = json.loads(resp["body"].read())
    return result["results"][0]["outputText"].strip()

# 3) Test
for q in [
    "What dry ingredients do I need?",
    "How long should I cook each pancake side?",
    "What can I serve with the pancakes?"
]:
    print(f"\n🔎 Q: {q}\n🟢 A: {rag(q)}")



🔎 Q: What dry ingredients do I need?
🟢 A: The dry ingredients you need are:
1 cup all-purpose flour
2 tablespoons sugar
1 teaspoon baking powder
1 teaspoon baking soda
1 pinch of salt

🔎 Q: How long should I cook each pancake side?
🟢 A: Two minutes

🔎 Q: What can I serve with the pancakes?
🟢 A: The model cannot find sufficient information to answer the question.
